In [1]:
import nfl_data_py as nfl
import pandas as pd

# Definimos el rango de temporadas que queremos analizar
seasons = list(range(2020, 2025))

# Descargamos datos de snaps (jugadas en campo) por jugador en cada partido
snap_data = nfl.import_snap_counts(seasons)

# Descargamos datos del roster oficial por temporada con nombres y posiciones
roster_data = nfl.import_seasonal_rosters(seasons)

In [2]:
# Revisamos las columnas para entender qué vamos a usar para hacer el merge
print("Columnas en snap_data:", snap_data.columns.tolist())
print("Columnas en roster_data:", roster_data.columns.tolist())

Columnas en snap_data: ['game_id', 'pfr_game_id', 'season', 'game_type', 'week', 'player', 'pfr_player_id', 'position', 'team', 'opponent', 'offense_snaps', 'offense_pct', 'defense_snaps', 'defense_pct', 'st_snaps', 'st_pct']
Columnas en roster_data: ['season', 'team', 'position', 'depth_chart_position', 'jersey_number', 'status', 'player_name', 'first_name', 'last_name', 'birth_date', 'height', 'weight', 'college', 'player_id', 'espn_id', 'sportradar_id', 'yahoo_id', 'rotowire_id', 'pff_id', 'pfr_id', 'fantasy_data_id', 'sleeper_id', 'years_exp', 'headshot_url', 'ngs_position', 'week', 'game_type', 'status_description_abbr', 'football_name', 'esb_id', 'gsis_it_id', 'smart_id', 'entry_year', 'rookie_year', 'draft_club', 'draft_number', 'age']


In [3]:
# Observamos los datos de ejemplo para confirmar las columnas clave: 
print("Ejemplos snap_data['player'](nombres jugadores):")
print(snap_data['player'].head())

print("Ejemplos roster_data['player_name'](nombres jugadores):")
print(roster_data['player_name'].head())

Ejemplos snap_data['player'](nombres jugadores):
0         Andrew Wylie
1    Mitchell Schwartz
2        Austin Reiter
3      Kelechi Osemele
4          Eric Fisher
Name: player, dtype: object
Ejemplos roster_data['player_name'](nombres jugadores):
0    Adam Vinatieri
1         Tom Brady
2        Drew Brees
3       Josh McCown
4      Jason Witten
Name: player_name, dtype: object


In [4]:
# Para unir los datos, usaremos los nombres de los jugadores:
alineacion = snap_data.merge(
    roster_data[['player_name', 'position', 'team', 'season']],
    left_on=['player', 'season', 'team'],
    right_on=['player_name', 'season', 'team'],
    how='left'
)

In [6]:
# El merge podría agregar sufijos si hay columnas con nombre igual
# Mostramos las columnas resultantes para identificar estos sufijos
print("Columnas después del merge:", alineacion.columns.tolist())

Columnas después del merge: ['game_id', 'pfr_game_id', 'season', 'game_type', 'week', 'player', 'pfr_player_id', 'position_x', 'team', 'opponent', 'offense_snaps', 'offense_pct', 'defense_snaps', 'defense_pct', 'st_snaps', 'st_pct', 'player_name', 'position_y']


In [10]:
# Seleccionamos las columnas importantes, adaptamos sufijos según aparezcan
alineacion = alineacion[[
    'game_id',
    'team',
    'player',
    'player_name',
    'position_y',
    'week',
    'season',
    'offense_snaps',
    'defense_snaps',
    'st_snaps'
]]

alineacion = alineacion.rename(columns={
    'player' : 'player_name_snap_data'
})

In [11]:
alineacion['starter'] = alineacion['offense_snaps'] > 0

print(alineacion.head())

          game_id team player_name_snap_data        player_name position_y  \
0  2020_01_HOU_KC   KC          Andrew Wylie       Andrew Wylie         OL   
1  2020_01_HOU_KC   KC     Mitchell Schwartz  Mitchell Schwartz         OL   
2  2020_01_HOU_KC   KC         Austin Reiter      Austin Reiter         OL   
3  2020_01_HOU_KC   KC       Kelechi Osemele    Kelechi Osemele         OL   
4  2020_01_HOU_KC   KC           Eric Fisher        Eric Fisher         OL   

   week  season  offense_snaps  defense_snaps  st_snaps  starter  
0     1    2020           69.0            0.0       6.0     True  
1     1    2020           69.0            0.0       6.0     True  
2     1    2020           69.0            0.0       6.0     True  
3     1    2020           69.0            0.0       6.0     True  
4     1    2020           69.0            0.0       6.0     True  


In [12]:
# Guardamos nuestro resultado en un CSV para analisis posterior
alineacion.to_csv('nfl_alineacion_2020_2024.csv', index=False)
print("Archivo 'nfl_alineacion_2020_2024.csv' guardado con éxito!")

Archivo 'nfl_alineacion_2020_2024.csv' guardado con éxito!
